In [ ]:


# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from PIL import Image as Img
import openslide
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns
from IPython.display import Image, display
import sklearn.preprocessing
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

from sklearn.model_selection import train_test_split
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import cv2
from tqdm.notebook import tqdm
import skimage.io
from skimage.transform import resize, rescale
from skimage.color import rgb2hed,hed2rgb
import os

from fastai.callbacks import SaveModelCallback
from fastai.callbacks import *

# import albumentations as A

import shutil
import zipfile

import time
import cv2
from copy import deepcopy

import fastai
from fastai.core import *
from fastai.vision import *
from fastai.vision import Image as fImage
from fastai.vision import pil2tensor
from fastai.metrics import *

import pickle
from torchvision import transforms
from matplotlib.colors import ListedColormap
# import collections
from collections import defaultdict, Counter
import torch
import torch.nn as nn
import torch.nn.functional as F
# from torchsummary import summary
from torch.utils.data import Dataset,DataLoader
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.tensorboard import SummaryWriter
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
from torch import autograd

from albumentations import Compose, Normalize, HorizontalFlip, VerticalFlip
from albumentations.pytorch import ToTensorV2


from torch.utils.data import Dataset,DataLoader
from sklearn.metrics import cohen_kappa_score
import scipy as sp
import random

import sys

sys.path = [
    '../input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master',
    '../input/nvidiaapex'
] + sys.path
from efficientnet_pytorch import model as enet

from apex import amp

# import torch_xla.core.xla_model as xm
# import torch_xla.distributed.parallel_loader as pl
# import torch_xla.distributed.xla_multiprocessing as xmp


In [ ]:
data_path = os.path.join('/kaggle/input','prostate-cancer-grade-assessment')

print(data_path)

In [ ]:


train_df = pd.read_csv(os.path.join(data_path,'train.csv'))
train_df.head()



In [ ]:
test_df = pd.read_csv(os.path.join(data_path,'test.csv'))

test_df.head()

In [ ]:


train_imgs_path = []
train_imgs_dir = os.path.join(data_path,'train_images')
for path in os.listdir(train_imgs_dir):
#     print(train_df[train_df['image_id'] == path[:-5]]['data_provider'].values[0])
#     if(train_df[train_df['image_id'] == path[:-5]]['data_provider'].values[0] =='radboud'):
    train_imgs_path.append(path)



In [ ]:


train_labels_path = []
train_labels_dir = os.path.join(data_path,'train_label_masks')
for path in os.listdir(train_labels_dir):
    train_labels_path.append(path)



In [ ]:


bad_samples = []
for train_path in train_imgs_path:
#     print(train_path[:-5]+"_mask.tiff")
    if train_path[:-5]+"_mask.tiff" not in train_labels_path:
        bad_samples.append(train_path)

print("Number of bad samples: ",len(bad_samples))



In [ ]:
len(train_df)

In [ ]:
train_df = train_df.set_index('image_id')
train_df = train_df.drop('ffe9bcababc858e04840669e788065a1')
train_df = train_df.reset_index()

In [ ]:
len(train_df)

In [ ]:
train_df.info()

In [ ]:
train_imgs_path = sorted(train_imgs_path)
train_labels_path = sorted(train_labels_path)

In [ ]:


def find_bad_samples(train_imgs_path):
    """
    Looks for samples that doesn't have corresponding mask. 
    
    param: training_imgs_path - path to directory containing images
    
    returns: array containing indexes of bad samples (bad = doesn't have label)
    """
    bad_samples = []
    for train_path in train_imgs_path:
#     print(train_path[:-5]+"_mask.tiff")
        if train_path[:-5]+"_mask.tiff" not in train_labels_path:
            bad_samples.append(train_path)
    return bad_samples



In [ ]:
def get_random_idx():
    return train_df.sample(1)['image_id'].values[0]

In [ ]:
def get_karolinska_idx():
    return train_df[train_df['data_provider']=='karolinska'].sample(1)['image_id'].values[0]

In [ ]:
train_df.head()

In [ ]:
def print_imgs_dirs():
    count = 0
    for file in os.listdir('/kaggle/input/preprocessingint-0-1000-rgb/train_imgs'):
        count +=1
    for file in os.listdir('/kaggle/input/preprocessingint-1000-2000-rgb/train_imgs'):
        count +=1
    for file in os.listdir('/kaggle/input/preprocessingint-2000-3000-rgb/train_imgs'):
        count +=1
    for file in os.listdir('/kaggle/input/preprocessingint-3000-4000-rgb/train_imgs'):
        count +=1
    for file in os.listdir('/kaggle/input/preprocessingint-4000-5000-rgb/train_imgs'):
        count +=1
    for file in os.listdir('/kaggle/input/preprocessingint-5000-6000-rgb/train_imgs'):
        count +=1
    for file in os.listdir('/kaggle/input/preprocessingint-6000-7000-rgb/train_imgs'):
        count +=1
    for file in os.listdir('/kaggle/input/preprocessingint-7000-8000-rgb/train_imgs'):
        count +=1
    for file in os.listdir('/kaggle/input/preprocessingint-8000-9000-rgb/train_imgs'):
        count +=1
    for file in os.listdir('/kaggle/input/preprocessingint-9000-10000-rgb/train_imgs'):
        count +=1
    for file in os.listdir('/kaggle/input/preprocessingint-10000-end-rgb/train_imgs'):
        count +=1



    print('Found ',count,' images')
print_imgs_dirs()

In [ ]:
def seed_torch(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_torch(seed=42)

In [ ]:
class PandaDataset(Dataset):
    def __init__(self, df,idx_list, transform=None):
        self.df = df
        self.idx_list = idx_list
        self.transform = transform
        self.paths = ['/kaggle/input/preprocessingint-0-1000-rgb/train_imgs',
                     '/kaggle/input/preprocessingint-1000-2000-rgb/train_imgs',
                     '/kaggle/input/preprocessingint-2000-3000-rgb/train_imgs',
                     '/kaggle/input/preprocessingint-3000-4000-rgb/train_imgs',
                     '/kaggle/input/preprocessingint-4000-5000-rgb/train_imgs',
                     '/kaggle/input/preprocessingint-5000-6000-rgb/train_imgs',
                     '/kaggle/input/preprocessingint-6000-7000-rgb/train_imgs',
                     '/kaggle/input/preprocessingint-7000-8000-rgb/train_imgs',
                     '/kaggle/input/preprocessingint-8000-9000-rgb/train_imgs',
                     '/kaggle/input/preprocessingint-9000-10000-rgb/train_imgs',
                     '/kaggle/input/preprocessingint-10000-end-rgb/train_imgs']
        
    def __len__(self):
        return len(self.idx_list)

    def __getitem__(self, idx):        
        idx = self.idx_list[idx]
        
        if idx >='0005f7aaab2800f6170c399693a96917' and idx <= '1995d7f1f42ad259aae1fc004fbc6360':
            path = self.paths[0]
        elif idx >= '19963a5a3c42675307056e0be36851cf' and idx <= '3208b512a881073da1fad575ae4e8cf3':
            path = self.paths[1]
        elif idx >= '320a5a8ed48d1d03103a79659913c987' and idx <= '4b2af68b112c6842e2fce65b9f69473a':
            path = self.paths[2]
        elif idx >= '4b38723b387a432075f7cb3544172900' and idx <= '636430b4495809e379c2623c22f95d59':
            path = self.paths[3]
        elif idx >= '636498ce17020805892b1dd50e5eed64' and idx <= '7b576fbb0da9890869714dadff5226df':
            path = self.paths[4]
        elif idx >= '7b5b2d7f86f7725b0ea4f0ed3b4807de' and idx <= '9359e8987975c5e0d24dcb18e41270dd':
            path = self.paths[5]
        elif idx >= '9362e969027cc9260ceabfcbe2a7ceda' and idx <= 'aa8ee3fd85494e92cc6595f97d42dc6a':
            path = self.paths[6]
        elif idx >= 'aa956e452824c715d48722bbe45c2882' and idx <= 'c176b834c3a5045ac074fb70637cfa8d':
            path = self.paths[7]
        elif idx >= 'c176d5630828984c57bf0c94751145a8' and idx <= 'd996d408cbc9200c7254ab6955e31959':
            path = self.paths[8]
        elif idx >= 'd997bd54e6a545c95817a62b77fb43df' and idx <= 'f1f74c4aecfa7f9baa760bb992e4c720':
            path = self.paths[9]
        elif idx >= 'f1f9aea9e0bb9845704f9871346811aa':
            path = self.paths[10]  
        
        final_path = os.path.join(path,idx)
        res = []
        
        for tile in os.listdir(final_path):
            tile_img = Img.open(os.path.join(final_path,tile))
            tile_img = np.asarray(tile_img)
            res.append(tile_img)
        
        rnd_perm = np.random.permutation(len(res))
        
        img = cv2.hconcat([
        cv2.vconcat([res[rnd_perm[0]],res[rnd_perm[1]],res[rnd_perm[2]],res[rnd_perm[3]],res[rnd_perm[4]],res[rnd_perm[5]]]),
        cv2.vconcat([res[rnd_perm[6]],res[rnd_perm[7]],res[rnd_perm[8]],res[rnd_perm[9]],res[rnd_perm[10]],res[rnd_perm[11]]]),
        cv2.vconcat([res[rnd_perm[12]],res[rnd_perm[13]],res[rnd_perm[14]],res[rnd_perm[15]],res[rnd_perm[16]],res[rnd_perm[17]]]),
        cv2.vconcat([res[rnd_perm[18]],res[rnd_perm[19]],res[rnd_perm[20]],res[rnd_perm[21]],res[rnd_perm[22]],res[rnd_perm[23]]]),
        cv2.vconcat([res[rnd_perm[24]],res[rnd_perm[25]],res[rnd_perm[26]],res[rnd_perm[27]],res[rnd_perm[28]],res[rnd_perm[29]]]),
        cv2.vconcat([res[rnd_perm[30]],res[rnd_perm[31]],res[rnd_perm[32]],res[rnd_perm[33]],res[rnd_perm[34]],res[rnd_perm[35]]])
    ])
        
        sample =self.df.loc[idx,:]
#         file_name = idx+".png"
        isup  = sample['isup_grade']
        
#         img = Img.open(os.path.join(path,file_name))
        
#         img = np.asarray(img)
        
        if self.transform:
            augmented = self.transform(image=img)
            img = augmented['image']
            
            
        label = np.zeros(5).astype(np.float32)
        label[:int(isup)] = 1.
#         isup = torch.tensor(isup).float()
        
        return img,torch.tensor(label)
        


def get_transforms_type(data='train'):
    
    assert data in ('train', 'valid')
    
    if data == 'train':
        return Compose([
            HorizontalFlip(p=0.5),
            VerticalFlip(p=0.5),
            Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])
    
    elif data == 'valid':
        return Compose([
            Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])


In [ ]:
os.listdir('/kaggle/input/efficientnet-pytorch')

In [ ]:
class PCAModel(nn.Module):
    def __init__(self,output_num):
        super().__init__()
        self.effic = enet.EfficientNet.from_name('efficientnet-b0')
        self.effic.load_state_dict(torch.load(os.path.join('/kaggle/input/efficientnet-pytorch','efficientnet-b0-08094119.pth')))
        self.input_dim = self.effic._fc.in_features
        self.head = nn.Linear(self.input_dim,output_num)
        self.effic._fc = nn.Identity()
    
    def extract(self,x):
        return self.effic(x)
            

    def forward(self,x):
        x = self.extract(x)
        x = self.head(x)
        return x
        


In [ ]:
train_df = pd.read_csv(os.path.join('/kaggle/input/panda-folds','folds.csv'))
train_df = train_df.set_index('image_id')
train_df = train_df.drop('ffe9bcababc858e04840669e788065a1')
train_df = train_df.reset_index()

In [ ]:
fold = 1
X_train = train_df[train_df['fold'] != fold]
X_val = train_df[train_df['fold'] == fold]
X_train = X_train.set_index('image_id')
X_val = X_val.set_index('image_id')

In [ ]:
X_train.head()


In [ ]:
X_val.head()


In [ ]:
X_train['isup_grade'].value_counts()/len(X_train)

In [ ]:
X_val['isup_grade'].value_counts()/len(X_val)

In [ ]:
os.makedirs('modelw',exist_ok=True)

In [ ]:
# trainSetSample = X_train.sample(200)
# valSetSample = X_val.sample(50)

trainSet = PandaDataset(X_train,X_train.index,transform=get_transforms_type(data='train'))
valSet = PandaDataset(X_val,X_val.index,transform=get_transforms_type(data='valid'))

batch_size = 5
dataloaders = {
    'train': DataLoader(trainSet, batch_size=batch_size, shuffle=True, num_workers=4),
    'val': DataLoader(valSet, batch_size=batch_size, shuffle=False, num_workers=4)
}

In [ ]:
Counter(X_train['isup_grade'])

In [ ]:
Counter(X_val['isup_grade'])

In [ ]:
len(X_train),len(X_val)

In [ ]:
def check_input():
    fig,ax = plt.subplots(1,2)
    for imgs,labels in dataloaders['val']:
        ax[0].imshow(imgs[0].permute(1,2,0))
        ax[0].set_title(labels[0])

        ax[1].imshow(imgs[1].permute(1,2,0))
        ax[1].set_title(labels[1])
        break
# check_input()    

In [ ]:
gc.collect()

In [ ]:
def accuracy_(y_hat,y):
    return np.mean(y_hat == y)

def quadratic_weighted_kappa(y_hat,y):
    return cohen_kappa_score(y_hat,y,weights='quadratic')


In [ ]:
def train_model(model,optimizer,scheduler,start_epoch=0,end_epoch=20,save_path='modelw',apex=False,checkpoint=None,best_score=None):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     LOGGER.debug('using device: ',device)
    
    model.to(device)
    
    if apex:
        print('USING APEX')
        model, optimizer = amp.initialize(model, optimizer, opt_level="O1",verbosity=0)
        if checkpoint is not None: 
            print('Loading checkpoint - APEX')
            model.load_state_dict(checkpoint['model'])
            optimizer.load_state_dict(checkpoint['optimizer'])
            amp.load_state_dict(checkpoint['amp'])
#     best_model_wts = deepcopy(model.state_dict())
    
    criterion = nn.BCEWithLogitsLoss()
    criterion2 = nn.MSELoss()
    
    if best_score is None:
        best_score = -100
    else:
        best_score = best_score
    best_loss = np.inf
    

    num_epochs = end_epoch - start_epoch
    for epoch in range(start_epoch,end_epoch):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        
        since = time.time()
        
        for phase in ['train','val']:
            if phase == 'train':
            
                model.train()
            
            else:
                model.eval()
                
#             tk = tqdm(enumerate(dataloaders[phase]),total=len(dataloaders[phase]))
            avg_loss = 0
            
            predictions = []
            valid_labels = []
            
            for (imgs,labels) in dataloaders[phase]:

                imgs = imgs.to(device)
                labels = labels.to(device)
                
#                 print('imgs shape: ',imgs.shape)
#                 print('lbaels shape: ',labels.shape)
                
                with torch.set_grad_enabled(phase=='train'):
                
                    preds = model(imgs)
                    
                
                loss = criterion(preds,labels)
                
#                     print('loss ',loss)
                        
                avg_loss += loss.item()/ len(dataloaders[phase])
                    
#                     print('avg loss: ',avg_loss)
                    
                preds = preds.sigmoid().sum(1).detach().round()
        
                predictions.append(preds.to('cpu'))
                valid_labels.append(labels.to('cpu').sum(1).detach().numpy())
                
                del labels,imgs
                gc.collect()
                    
                if phase == 'train':
                    
                    if apex:
                        with amp.scale_loss(loss, optimizer) as scaled_loss:
                            scaled_loss.backward()
                    else:
                        loss.backward()
                    optimizer.step()
                    optimizer.zero_grad()
                        
                     
            if phase =='val':
                scheduler.step(avg_loss)
                        
            preds = np.concatenate(predictions)
            valid_labels = np.concatenate(valid_labels) 
            
                        
            final_preds = preds
            score = quadratic_weighted_kappa(valid_labels, final_preds)
#                 
            acc = accuracy_score(valid_labels,final_preds)
                
            time_elapsed = time.time() - since
            print(f"phase: {phase}, loss:{avg_loss}, kappa_score: {score}, acc: {acc}")
            print('Time: {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
                        
                
            if phase == 'val':
                if score>best_score:
                    best_score = score
#                     best_model_wts = deepcopy(model.state_dict())
                    print(f'  Epoch {epoch} - Save Best Score: {best_score:.4f}')
                    if apex:
                        checkpoint = {
                            'model': model.state_dict(),
                            'optimizer': optimizer.state_dict(),
                            'amp': amp.state_dict()
                        }
                        torch.save(checkpoint, os.path.join(save_path,'amp_checkpoint.pt'))
                    else:
                        torch.save(model.state_dict(), os.path.join(save_path,f'panda_callback.pt'))
                    

                        
#     model.load_state_dict(best_model_wts)
#     torch.save(model,os.path.join(save_path,'fullModel.pt'))
            
    return model
            
            

In [ ]:

gc.collect()

In [ ]:
lr = 1e-3
model = PCAModel(5)
# model.load_state_dict(torch.load(os.path.join('/kaggle/input/pandainterefficregr','panda_callback.pt')))

optimizer = optim.Adam([
     {'params':model.effic.parameters(),'lr':lr/10},
    {'params':model.head.parameters()}
] ,lr=lr, amsgrad=False)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=3, verbose=True, eps=1e-6)

checkpoint =  torch.load(os.path.join('/kaggle/input/eksperyment5fold1','amp_checkpoint.pt'))
# checkpoint = None
# scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.01, steps_per_epoch=len(dataloaders['train']), epochs=10)

model,training_stats,coefficients =  train_model(model,optimizer,scheduler,start_epoch=14,end_epoch=20,apex=True,checkpoint=checkpoint)

In [ ]:
#adawsaawdsassa